<a href="https://colab.research.google.com/github/Soumya4523/Image-Captioning-using-Attention-Mechanism/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.callbacks import Callback
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Embedding,LSTM


In [11]:
df = pd.read_csv(r"/content/IMDB Dataset.csv", nrows=100)
df["sentiment"].replace({"positive": 1, "negative": 0}, inplace=True)

x = np.array(df["review"].values)
y = np.array(df["sentiment"].values)

x_filtered = []

for review in x:

    #lowercasing the sentence
    review = review.lower()

    # removing punctuations from sentence
    for i in review:
        punc = '''  !()-[]{};:'"\,<>./?@#$%^&*_~  '''
        if i in punc :
            review = review.replace(i, " ")

    x_filtered.append(review)

print("Data Preparation Stage-1 completed !")

Data Preparation Stage-1 completed !


In [12]:
# One-Hot Encoding each sentence
vocalbulary_size = 5000
onehot_encoded = [one_hot(review,vocalbulary_size) for review in x_filtered]

# Padding each encoded sentence to have a max_length=500
max_length=500
x_padded = pad_sequences(onehot_encoded,max_length,padding="post")

x_train,x_test,y_train,y_test = train_test_split(x_padded,y,test_size=0.2)

print("Data Preparation Stage-2 completed !")

Data Preparation Stage-2 completed !


In [13]:
model = Sequential()
embeded_vector_size = 35
model.add(Embedding(vocalbulary_size,embeded_vector_size,input_length=max_length))
model.add(LSTM(100))
model.add(Dense(1,activation="sigmoid"))

model.compile(optimizer='adam', loss='binary_crossentropy',metrics=["accuracy"])

print(model.summary())
print("Model Creation Completed !")



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 35)           175000    
                                                                 
 lstm (LSTM)                 (None, 100)               54400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 229501 (896.49 KB)
Trainable params: 229501 (896.49 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Model Creation Completed !


In [15]:
model.fit(x_train,y_train,epochs=10)

Epoch 1/10
3/3 [==============================] - 1s 495ms/step - loss: 0.6868 - accuracy: 0.6125
Epoch 2/10
3/3 [==============================] - 2s 504ms/step - loss: 0.6813 - accuracy: 0.6125
Epoch 3/10
3/3 [==============================] - 2s 525ms/step - loss: 0.6764 - accuracy: 0.6125
Epoch 4/10
3/3 [==============================] - 1s 335ms/step - loss: 0.6785 - accuracy: 0.6250
Epoch 5/10
3/3 [==============================] - 1s 325ms/step - loss: 0.6730 - accuracy: 0.6250
Epoch 6/10
3/3 [==============================] - 1s 305ms/step - loss: 0.6814 - accuracy: 0.6250
Epoch 7/10
3/3 [==============================] - 1s 311ms/step - loss: 0.6731 - accuracy: 0.6250
Epoch 8/10
3/3 [==============================] - 1s 309ms/step - loss: 0.6731 - accuracy: 0.6250
Epoch 9/10
3/3 [==============================] - 1s 305ms/step - loss: 0.6709 - accuracy: 0.6250
Epoch 10/10
3/3 [==============================] - 1s 319ms/step - loss: 0.6674 - accuracy: 0.6250


In [16]:
score = model.evaluate(x_test,y_test)
print(score)

1/1 [==============================] - 1s 601ms/step - loss: 0.6357 - accuracy: 0.7500
[0.6356683969497681, 0.75]


In [17]:
model.save("IMDB_sentiment_analysis")

In [26]:
from keras.models import load_model

trained_model = load_model(r"/content/IMDB_sentiment_analysis")
predicted = trained_model.predict(x_test)[2]

sentiment = 1 if predicted > 0.6 else 0

print("PREDICTED : ",sentiment)
print("ACTUAL : ",y_test[2])


1/1 [==============================] - 1s 745ms/step
PREDICTED :  0
ACTUAL :  0


In [23]:
def get_sentiment(sentence: str):
    if isinstance(sentence, (str)):
        pass
    else:
        raise Exception("Input needs to be of type 'str' ")

    # filtering the sentence
    sentence = sentence.lower()

    punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''

    for word in sentence:
        if word in punc:
            sentence = sentence.replace(word, " ")

    # Loading the saved trained model.
    from keras.models import load_model

    trained_model = load_model(r"/content/IMDB_sentiment_analysis")

    predicted = trained_model.predict(x_test)[2]
    sentiment = 1 if predicted >= 0.6 else 0

    if sentiment == 1:
        print("Positive")
    else:
        print("Negative")

    return sentiment


get_sentiment("That movie was really good!")

1/1 [==============================] - 1s 509ms/step
Negative


0